In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import os
import json 
import timm
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A

# Label Encoder

In [2]:
# 변수 설명 csv 파일 참조
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

In [3]:
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

# ============= add
labels = pd.read_csv('./data/train.csv')

train_label_encoder = {}
label_cnt = 0
previous_label = '0_00_0'
for i, label in enumerate(tqdm(sorted(labels['label']))) :
    crop_val = label.split('_')[0] # crop
    disease_val = label.split('_')[1] # disease
    risk_val = label.split('_')[2] # risk
    
    tmp_label = f'{crop_val}_{disease_val}_{risk_val}'
    if previous_label != tmp_label :
        train_label_encoder[tmp_label] = label_cnt
        previous_label = tmp_label
        label_cnt += 1
        
train_label_decoder = {val : key for key, val in train_label_encoder.items()}
display(train_label_decoder)
display(train_label_encoder)

100%|██████████████████████████████████████████████████████████████████████████| 5767/5767 [00:00<00:00, 961618.48it/s]


{0: '1_00_0',
 1: '2_00_0',
 2: '2_a5_2',
 3: '3_00_0',
 4: '3_a9_1',
 5: '3_a9_2',
 6: '3_a9_3',
 7: '3_b3_1',
 8: '3_b6_1',
 9: '3_b7_1',
 10: '3_b8_1',
 11: '4_00_0',
 12: '5_00_0',
 13: '5_a7_2',
 14: '5_b6_1',
 15: '5_b7_1',
 16: '5_b8_1',
 17: '6_00_0',
 18: '6_a11_1',
 19: '6_a11_2',
 20: '6_a12_1',
 21: '6_a12_2',
 22: '6_b4_1',
 23: '6_b4_3',
 24: '6_b5_1'}

{'1_00_0': 0,
 '2_00_0': 1,
 '2_a5_2': 2,
 '3_00_0': 3,
 '3_a9_1': 4,
 '3_a9_2': 5,
 '3_a9_3': 6,
 '3_b3_1': 7,
 '3_b6_1': 8,
 '3_b7_1': 9,
 '3_b8_1': 10,
 '4_00_0': 11,
 '5_00_0': 12,
 '5_a7_2': 13,
 '5_b6_1': 14,
 '5_b7_1': 15,
 '5_b8_1': 16,
 '6_00_0': 17,
 '6_a11_1': 18,
 '6_a11_2': 19,
 '6_a12_1': 20,
 '6_a12_2': 21,
 '6_b4_1': 22,
 '6_b4_3': 23,
 '6_b5_1': 24}

# Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, files, transforms):
        self.files = files
        self.label_encoder = train_label_encoder #label_encoder
        
        self.deit_transforms = transforms[0]
        self.effi_transforms = transforms[1]
                
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('\\')[-1]
        
        
        # image
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        deit_img = self.deit_transforms(image=img)["image"]
        effi_img = self.effi_transforms(image=img)["image"]
        
        deit_img = deit_img.transpose(2,0,1)
        effi_img = effi_img.transpose(2,0,1)
        
        return {
            'deit_img' : torch.tensor(deit_img, dtype=torch.float32)/ 255.0,
            'effi_img' : torch.tensor(effi_img, dtype=torch.float32)/ 255.0,
        }

# Model

In [5]:
import timm
import torch.nn as nn

class ViT(nn.Module):
    def __init__(self):
        super(ViT, self).__init__()
        self.model = timm.create_model('vit_small_patch16_224', num_classes=25, pretrained=True)
        
    def forward(self, inputs):
        output = self.model(inputs)
        return output
    
    
    
class pretrained_DeiT(nn.Module):
    def __init__(self):
        super(pretrained_DeiT, self).__init__()
        self.model = timm.create_model('deit_small_patch16_224', num_classes=38, pretrained=False)

    def forward(self, inputs):
        output = self.model(inputs)
        return output

class pretrained_EffiV2S(nn.Module):
    def __init__(self):
        super(pretrained_EffiV2S, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_s', num_classes=38, pretrained=False)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

    
    
class Deit(nn.Module) :
    def __init__(self, model_path=None, test=False) :
        super(Deit, self).__init__()
        self.pre_model = pretrained_DeiT()
        if test == False :
            self.pre_model.load_state_dict(torch.load(model_path, map_location=device))
        
        self.fc = nn.Linear(38, 25)
        
    def forward(self, x) :
        x = self.pre_model(x)
        x = self.fc(x)
        return x
    
class EffiV2S(nn.Module) :
    def __init__(self, model_path=None, test=False) :
        super(EffiV2S, self).__init__()
        self.pre_model = pretrained_EffiV2S()
        if test == False :
            self.pre_model.load_state_dict(torch.load(model_path, map_location=device))
        
        self.fc = nn.Linear(38, 25)
        
    def forward(self, x) :
        x = self.pre_model(x)
        x = self.fc(x)
        return x    

# Hyperparameter

In [6]:
device = torch.device("cuda:0")#("cpu")
batch_size = 16

In [7]:
deit_transforms = A.Compose([
    A.Resize(224, 224)
])

effi_transforms = A.Compose([
    A.Resize(288, 288)
])

test = sorted(glob('data/test/*'))
test_dataset = CustomDataset(test, (deit_transforms, effi_transforms))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Prediction

In [8]:
import torch.nn.functional as F

def softvoting(models, img, n_classes=25) :

    predicts = torch.zeros(img.size(0), n_classes)
    with torch.no_grad() :
        for model in models :
            output = model(img)
            output = F.softmax(output.cpu(), dim=1)
            predicts += output

    return predicts.detach().cpu() / len(models)


def _softvoting(models, img, n_classes=25) :

    predicts = torch.zeros(img.size(0), n_classes)
    with torch.no_grad() :
        for model in models :
            output = model(img)
            output = F.softmax(output.cpu(), dim=1)
            predicts += output

    return predicts.detach().cpu()

In [9]:
def predict(dataset, models, model_name) :
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset :
        if model_name == 'effi' :
            img = batch_item['effi_img'].to(device)

        elif model_name == 'deit' :
            img = batch_item['deit_img'].to(device)
            
        predictions = softvoting(models, img)
#         print(predictions.shape)
        batch_result = [int(torch.argmax(prediction)) for prediction in predictions]
        
        results.extend(predictions)
#         results.append(predictions)

    return results


def _predict(dataset, models, model_name) :
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset :
        if model_name == 'effi' :
            img = batch_item['effi_img'].to(device)

        elif model_name == 'deit' :
            img = batch_item['deit_img'].to(device)
            
        predictions = _softvoting(models, img)
#         print(predictions.shape)
#         batch_result = [int(torch.argmax(prediction)) for prediction in predictions]
        
#         results.extend(predictions)
        results.append(predictions)

    return results

## K fold models

In [13]:
kfold_models_path = glob('./model/k_fold_50k_pretrained_deit/*.pt')
models = []
for kfold_model_path in kfold_models_path :
    model = Deit(model_path)
#     model = EffiV2S(model_path)
    model.load_state_dict(torch.load(kfold_model_path, map_location=device))
    model.to(device).eval()
    models.append(model)

    
preds = predict(test_dataloader, models)

preds_cp = preds

preds_cp = np.array([train_label_decoder[int(val)] for val in preds_cp])
submission_csv = pd.read_csv('./data/sample_submission.csv')
submission_csv['label'] = preds_cp
submission_csv.to_csv('./data/kfold_public_vill_50k_pretrain_Deit_S16patch224_25E.csv', index=False)

## single model

In [14]:
# single_model_path = './model/k_fold_50k_pretrained_effiv2S/4_f94_public_vill_50k_pretrain_efficientnetv2S.pt'
# model = EffiV2S(model_path)
# model.load_state_dict(torch.load(single_model_path, map_location=device))
# model.to(device).eval()

# preds = predict(test_dataloader, model)

# preds_cp = preds

# preds_cp = np.array([train_label_decoder[int(val)] for val in preds_cp])
# submission_csv = pd.read_csv('./data/sample_submission.csv')
# submission_csv['label'] = preds_cp
# submission_csv.to_csv('./data/4fold_public_vill_50k_pretrain_efficientnetv2S.csv', index=False)

## 2 - kind of models
- effiv2(2) + deit(2) = 4 models

In [15]:
# effi_model_path = ['./model/k_fold_50k_pretrained_effiv2S/4_f9462_public_vill_50k_pretrain_efficientnetv2S.pt',
#                './model/k_fold_50k_pretrained_effiv2S/1_f9222_public_vill_50k_pretrain_efficientnetv2S.pt']

# deit_model_path = ['./model/k_fold_50k_pretrained_deit/1_f9164_public_vill_50k_pretrain_Deit_S16patch224_25E.pt',
#               './model/k_fold_50k_pretrained_deit/4_f9082_public_vill_50k_pretrain_Deit_S16patch224_25E.pt']

# ensemble_models = [effi_model_path[0], effi_model_path[1], deit_model_path[0], deit_model_path[1]]

# effi_models = []
# deit_models = []
# for ensemble_model_path in ensemble_models :
    
#     if 'efficient' in ensemble_model_path :
#         model = EffiV2S(test=True)
#         model.load_state_dict(torch.load(ensemble_model_path, map_location=device))
#         model.to(device).eval()
#         effi_models.append(model)
        
#     elif 'Deit' in ensemble_model_path :
#         model = Deit(test=True)
#         model.load_state_dict(torch.load(ensemble_model_path, map_location=device))
#         model.to(device).eval()
#         deit_models.append(model)
    


    
# effi_preds = _predict(test_dataloader, effi_models, model_name='effi')
# deit_preds = _predict(test_dataloader, deit_models, model_name='deit')

# effi_preds_cp = effi_preds
# deit_preds_cp = deit_preds


# # display(len(effi_preds_cp))
# # display(effi_preds_cp[0].shape)

# # display(len(deit_preds_cp))
# # display(deit_preds_cp[0].shape)


# results = []

# for effi, deit in zip(effi_preds_cp, deit_preds_cp) :
#     sum_pred = (effi + deit) / 4
    
#     batch_result = [int(torch.argmax(prediction)) for prediction in sum_pred]    
    
#     results.extend(batch_result)
# # print(len(results))
# predictions = np.array([train_label_decoder[int(val)] for val in results])
# submission_csv = pd.read_csv('./data/sample_submission.csv')
# submission_csv['label'] = predictions
# submission_csv.to_csv('./data/effi_f9462f9222_deit_f9164f9082.csv', index=False)

3245it [11:54,  4.54it/s]
3245it [09:30,  5.69it/s]


In [ ]:
preds_cp = np.array([train_label_decoder[int(val)] for val in preds_cp])
submission_csv = pd.read_csv('./data/sample_submission.csv')
submission_csv['label'] = preds_cp
submission_csv.to_csv('./data/effi_f9462f9222_deit_f9164f9082.csv', index=False)